In [1]:
!nvidia-smi

Wed Jul 21 16:30:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.42       Driver Version: 462.42       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   51C    P0    28W /  N/A |   1519MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x28fcd3fb5e0>)

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [8]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.0605, 0.1494, 0.4296],
        [0.4695, 0.6880, 0.6947]], device='cuda:0')

In [10]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [11]:
Y + Z

tensor([[1.0605, 1.1494, 1.4296],
        [1.4695, 1.6880, 1.6947]], device='cuda:0')

In [12]:
Z.cuda(0) is Z

True

In [13]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[0.3464],
        [0.3464]], device='cuda:0', grad_fn=<AddmmBackward>)

In [14]:
net[0].weight.data.device

device(type='cuda', index=0)